# Text Emotion Classification Analysis

In [5]:
!pip install tensorflow scikit-learn


In [6]:
!pip install tensorflow scikit-learn pandas numpy


In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Load the data
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())


                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


This is the problem of natural language processing, Start by tokenizing the data:

In [37]:
# Convert the columns to lists
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

Now we need to pad the sequence to the same length to feed them into a neural network

In [38]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)


Now use label encoder method to convert the classes from string to a numerical representation:

In [39]:
# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

We are now going to One-hot encode the labels. One hot encoding refers to the transformation of categorical labels into a binary representation where each label is represented as a vector of all zeros except a single 1. This is necessary because machine learning algorithms work with numerical data. So here is how we can One-hot encode the labels:

In [40]:
# One-hot encode the labels
one_hot_labels = tf.keras.utils.to_categorical(encoded_labels)


# Text Emotions Classififcation Model
Now split the data into training and test set:


In [41]:
# Split data into training and test sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, one_hot_labels, test_size=0.2)

Now let’s define a neural network architecture for our classification problem and use it to train a model to classify emotions:

In [48]:
# Define the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1,
                                    output_dim=128, input_length=max_length))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation="relu"))
model.add(tf.keras.layers.Dense(units=len(one_hot_labels[0]), activation="softmax"))

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))


Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.3839 - loss: 1.5259 - val_accuracy: 0.6969 - val_loss: 0.8740
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8462 - loss: 0.4931 - val_accuracy: 0.8144 - val_loss: 0.5573
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9807 - loss: 0.0714 - val_accuracy: 0.8222 - val_loss: 0.5863
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.9943 - loss: 0.0240 - val_accuracy: 0.8225 - val_loss: 0.6386
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.9970 - loss: 0.0147 - val_accuracy: 0.8209 - val_loss: 0.6815
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9986 - loss: 0.0094 - val_accuracy: 0.8197 - val_loss: 0.6920
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9975 - loss: 0.0107 - val_accuracy: 0.8128 - val_loss: 0.7680
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9981 - loss: 0.0077 - val_accu

 let’s take a sentence as an input text and see how the model performs:

In [72]:
input_text = "Its good see you"

#Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label= label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
['joy']
